In [1]:
import pandas as pd
import numpy as np

In [2]:
import pycountry

In [5]:
# Scores dataset output folder
DATASET_OUTPUT_DIR = "../dataset/"

# Dataset for visualization output folder
VIZ_DATA_DIR = "../03_visualization/"

### TIMES HIGHER EDUCATION SCORES

In [3]:
the_df = pd.read_csv('../data_raw/THE_World_Rankings.csv', index_col=0)
the_score_lab = 'scores_overall'

In [4]:
the_df.head(3)

,location,name,nid,rank,scores_citations,scores_citations_rank,scores_industry_income,scores_industry_income_rank,scores_international_outlook,scores_international_outlook_rank,...,scores_overall_rank,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,year,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio
0,United States,Harvard University,466,1,98.8,8,34.5,105,72.4,49,...,1,98.7,2,99.7,1,2011,NaN,NaN,NaN,NaN
1,United States,California Institute of Technology,128779,2,99.9,1,83.7,24,54.6,93,...,2,98.0,4,97.7,4,2011,NaN,NaN,NaN,NaN
2,United States,Massachusetts Institute of Technology,471,3,99.9,2,87.5,21,82.3,36,...,3,91.4,11,97.8,3,2011,NaN,NaN,NaN,NaN


In [5]:
the_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10980 entries, 0 to 10979
Data columns (total 21 columns):
location                             10980 non-null object
name                                 10980 non-null object
nid                                  10980 non-null int64
rank                                 10980 non-null object
scores_citations                     10530 non-null float64
scores_citations_rank                10980 non-null int64
scores_industry_income               10530 non-null object
scores_industry_income_rank          10980 non-null int64
scores_international_outlook         10530 non-null object
scores_international_outlook_rank    10980 non-null int64
scores_overall                       10530 non-null object
scores_overall_rank                  10980 non-null int64
scores_research                      10530 non-null float64
scores_research_rank                 10980 non-null int64
scores_teaching                      10530 non-null float64
scores_te

In [6]:
# Convert Overall score to numeric
the_df.loc[:, 'scores_overall_num'] = pd.to_numeric(the_df.scores_overall, errors='coerce').round(2)

In [7]:
the_df.groupby('year').agg({
    'scores_overall_num': lambda x: x.isna().sum(),
    'nid': ['count', 'nunique']
})

scores_overall_num   nid        
               <lambda> count nunique
year                                 
2011                0.0   200     200
2012              202.0   402     402
2013              200.0   400     400
2014              200.0   400     400
2015              200.0   401     401
2016                0.0   800     800
2017              780.0   981     981
2018              903.0  1103    1103
2019             1058.0  1258    1258
2020             1195.0  1397    1397
2021             1324.0  1526    1526
2022             1911.0  2112    2112

#### FIX AND RE-CALCULATE OVERALL SCORES

In [8]:
# Weights of scores in total score
the_weights = {
    'labs': ['scores_research', 'scores_teaching', 'scores_citations', 'scores_international_outlook', 'scores_industry_income'],
    2011: [30, 30, 32.5, 5, 2.5],
    2012: [30, 30, 30, 7.5, 2.5]
}

In [9]:
# Replace '-' with NANs and convert to floats
the_df.loc[ :,
    ['scores_international_outlook', 'scores_industry_income']
] = the_df.loc[:, ['scores_international_outlook', 'scores_industry_income']].replace('-', np.nan)

the_df.loc[:, the_weights['labs']] = the_df.loc[:, the_weights['labs']].astype('float32').round(2)

In [10]:
# Fix 2016 scores_overall having zero (0) values by replacing to NANs
the_df.loc[
    (the_df.year==2016)&(the_df.scores_overall_num==0),
    'scores_overall_num'
] = np.nan

the_df.loc[:, 'scores_overall_calc'] = np.nan

for year in np.unique(the_df.year):
    
    sub_df = the_df.loc[
        the_df.year==year,
        the_weights['labs']
    ]
    
    res = np.ma.average(
        np.ma.array(sub_df, mask=sub_df.isnull()),
        axis=1,
        weights=the_weights.get(year, the_weights[2012])
    )
    
    the_df.loc[
        the_df.year==year,
        'scores_overall_calc'
    ] = np.round(res.data, 2)

In [11]:
# Check calculations disrepancy
the_df.groupby('year').apply(
    lambda tx: (tx.scores_overall_num - tx.scores_overall_calc).abs().median()
)

year
2011    0.03
2012    0.03
2013    0.03
2014    0.03
2015    0.03
2016    0.03
2017    0.02
2018    0.02
2019    0.03
2020    0.02
2021    0.03
2022    0.03
dtype: float64

In [12]:
# Replace scores_overall NAN values with calculated scores
the_df.loc[
    the_df.scores_overall_num.isnull(),
    'scores_overall_num'
] = the_df.loc[ the_df.scores_overall_num.isnull(), 'scores_overall_calc' ]

In [13]:
# Save scores dataset to csv
#the_df.loc[
#    :,
#    ['location', 'name', 'nid', 'year', 'scores_overall_num'] + the_weights['labs']
#].to_csv(DATASET_OUTPUT_DIR + 'the_scores.csv', float_format = '%.2f')

### SHANGHAI RANKINGS SCORES

In [14]:
arwu_df = pd.read_csv('../data_raw/ARWU_World_Rankings.csv', index_col=0)
arwu_score_lab = 'score'

In [15]:
arwu_df.head(3)

,Alumni,Award,HiCi,N&S,PCP,PUB,ranking,region,regionLogo,regionRanking,score,univNameEn,univUp,year
0,NaN,100.0,100.0,100.0,68.7,100.0,1,United States,us,1,100.0,Harvard University,harvard-university,2003
1,NaN,76.2,88.2,73.8,80.5,72.2,2,United States,us,2,83.5,Stanford University,stanford-university,2003
2,NaN,72.9,68.0,64.1,100.0,52.0,3,United States,us,3,76.3,California Institute of Technology,california-institute-of-technology,2003


In [16]:
arwu_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11816 entries, 0 to 999
Data columns (total 14 columns):
Alumni           11315 non-null float64
Award            11815 non-null float64
HiCi             11815 non-null float64
N&S              11742 non-null float64
PCP              11815 non-null float64
PUB              11815 non-null float64
ranking          11816 non-null object
region           11816 non-null object
regionLogo       11816 non-null object
regionRanking    11816 non-null object
score            1904 non-null float64
univNameEn       11816 non-null object
univUp           10802 non-null object
year             11816 non-null int64
dtypes: float64(7), int64(1), object(6)
memory usage: 1.4+ MB


In [17]:
# Check overall scores NA values and duplicate universities
arwu_df.groupby('year').agg({
    'score': lambda x: x.isna().sum(),
    'univNameEn': ['count', 'nunique', 'size'],
})

score univNameEn              
     <lambda>      count nunique  size
year                                  
2003    399.0        500     499   500
2004    402.0        502     502   502
2005    400.0        500     500   500
2006    399.0        500     500   500
2007    409.0        510     510   510
2008    403.0        503     503   503
2009    401.0        501     501   501
2010    400.0        500     500   500
2011    399.0        500     500   500
2012    400.0        500     500   500
2013    400.0        500     500   500
2014    400.0        500     500   500
2015    400.0        500     500   500
2016    400.0        500     500   500
2017    700.0        800     800   800
2018    900.0       1000    1000  1000
2019    900.0       1000    1000  1000
2020    900.0       1000    1000  1000
2021    900.0       1000    1000  1000

#### What is duplicate university name from 2003?

In [18]:
arwu_df.loc[
    arwu_df.year==2003,
    'univNameEn'
].value_counts().head(3)

University of Duisburg-Essen    2
Emory University                1
The University of Adelaide      1
Name: univNameEn, dtype: int64

In [19]:
arwu_df.loc[
    (arwu_df.year==2003)&(arwu_df.univNameEn == 'University of Duisburg-Essen')
]

,Alumni,Award,HiCi,N&S,PCP,PUB,ranking,region,regionLogo,regionRanking,score,univNameEn,univUp,year
483,NaN,0.0,0.0,3.5,8.0,28.4,451-500,Germany,de,39-43,NaN,University of Duisburg-Essen,university-of-duisburg-essen-1,2003
484,NaN,0.0,0.0,2.0,7.9,29.6,451-500,Germany,de,39-43,NaN,University of Duisburg-Essen,university-of-duisburg-essen-2,2003


#### 'Alumni' scores from 2003 are NAs

In [20]:
arwu_df.loc[
    arwu_df.Alumni.isnull()
].head(3)

,Alumni,Award,HiCi,N&S,PCP,PUB,ranking,region,regionLogo,regionRanking,score,univNameEn,univUp,year
0,NaN,100.0,100.0,100.0,68.7,100.0,1,United States,us,1,100.0,Harvard University,harvard-university,2003
1,NaN,76.2,88.2,73.8,80.5,72.2,2,United States,us,2,83.5,Stanford University,stanford-university,2003
2,NaN,72.9,68.0,64.1,100.0,52.0,3,United States,us,3,76.3,California Institute of Technology,california-institute-of-technology,2003


In [21]:
arwu_df.loc[
    arwu_df.Alumni.isnull()
].tail(3)

,Alumni,Award,HiCi,N&S,PCP,PUB,ranking,region,regionLogo,regionRanking,score,univNameEn,univUp,year
498,NaN,0.0,0.0,3.2,8.0,28.9,451-500,Spain,es,8-13,NaN,University of Zaragoza,university-of-zaragoza,2003
499,NaN,0.0,0.0,6.7,6.5,26.9,451-500,United States,us,155-161,NaN,West Virginia University,west-virginia-university,2003
587,NaN,NaN,NaN,NaN,NaN,NaN,501-600,Finland,fi,4-6,NaN,Tampere University,tampere-university,2020


#### WEIGHTS

In [22]:
arwu_weights = {
    'labs': ['Alumni', 'Award', 'HiCi', 'PUB', 'PCP'],
    'wNS': [10, 20, 20, 20, 10, 20],
    'woNS': [10*1.25, 20*1.25, 20*1.25, 20*1.25, 10*1.25]
}

#### Drop NAs

In [23]:
# Drop 2003 year, it was first and the only year without 'Alumni' scores, methodoly for this year is not clear
arwu_df = arwu_df.loc[
    arwu_df.year!=2003
]

# Drop the only row with no any scores provided
arwu_df = arwu_df.loc[
    ~arwu_df[arwu_weights['labs']].isna().all(axis=1)
]

In [24]:
arwu_df.isna().sum()

Alumni              0
Award               0
HiCi                0
N&S                73
PCP                 0
PUB                 0
ranking             0
region              0
regionLogo          0
regionRanking       0
score            9512
univNameEn          0
univUp            946
year                0
dtype: int64

#### RE-CALCULATE OVERALL SCORES

In [25]:
arwu_df.loc[:, 'scores_overall_num'] = np.nan

# Universities with N&S score provided
sub_df = arwu_df.loc[
    ~arwu_df['N&S'].isnull(),
    arwu_weights['labs'] + ['N&S']
]

arwu_df.loc[
    ~arwu_df['N&S'].isnull(),
    'scores_overall_num'
] = np.average(sub_df, axis=1, weights=arwu_weights['wNS'])

# Universities without N&S score provided
sub_df = arwu_df.loc[
    arwu_df['N&S'].isnull(),
    arwu_weights['labs']
]

arwu_df.loc[
    arwu_df['N&S'].isnull(),
    'scores_overall_num'
] = np.average(sub_df, axis=1, weights=arwu_weights['woNS'])

In [26]:
# The highest scoring institution is assigned a score of 100,
# and other institutions are calculated as a percentage of the top score. (ARWU methodology)

arwu_df.loc[:, 'scores_overall_calc'] = arwu_df.groupby('year').apply(
    lambda tx: (tx.scores_overall_num / tx.scores_overall_num.max() * 100).round(1)
).values

In [27]:
# Check calculations disrepancy
arwu_df.groupby('year').apply(
    lambda tx: (tx.score - tx.scores_overall_calc).abs().mean(skipna=True)
)

year
2004    0.007000
2005    0.012000
2006    0.006931
2007    0.007921
2008    0.009000
2009    0.013000
2010    0.006000
2011    0.008911
2012    0.012000
2013    0.012000
2014    0.009000
2015    0.009000
2016    0.011000
2017    0.008000
2018    0.008000
2019    0.011000
2020    0.009000
2021    0.009000
dtype: float64

In [28]:
arwu_df.head()

,Alumni,Award,HiCi,N&S,PCP,PUB,ranking,region,regionLogo,regionRanking,score,univNameEn,univUp,year,scores_overall_num,scores_overall_calc
0,98.6,100.0,100.0,100.0,60.6,100.0,1,United States,us,1,100.0,Harvard University,harvard-university,2004,95.92,100.0
1,41.2,72.2,96.1,75.2,68.1,72.3,2,United States,us,2,77.2,Stanford University,stanford-university,2004,74.09,77.2
2,100.0,93.4,56.6,58.5,73.2,70.2,3,United Kingdom,gb,1,76.2,University of Cambridge,university-of-cambridge,2004,73.06,76.2
3,70.0,76.0,74.1,75.6,45.1,72.7,4,United States,us,3,74.2,"University of California, Berkeley",university-of-california-berkeley,2004,71.19,74.2
4,74.1,78.9,73.6,69.1,47.5,64.6,5,United States,us,4,72.4,Massachusetts Institute of Technology (MIT),massachusetts-institute-of-technology-mit,2004,69.40,72.4


In [29]:
# Save .csv
#arwu_df.loc[
#    :,
#    ['region', 'univNameEn', 'univUp', 'year', 'scores_overall_calc'] + arwu_weights['labs'] + ['N&S']
#].to_csv(DATASET_OUTPUT_DIR + 'arwu_scores.csv')

### QS RANKINGS SCORES

In [30]:
qs_df = pd.read_csv('../data_raw/QS_World_Rankings.csv', index_col=0)
qs_score_lab = 'Overall Score'

In [31]:
qs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10047 entries, 0 to 10046
Data columns (total 18 columns):
City                              9814 non-null object
Natural Sciences                  865 non-null float64
Life Sciences And Medicine        3030 non-null float64
Engineering And Technology        3030 non-null float64
Arts And Humanities               3041 non-null float64
International Students            7466 non-null float64
Social Sciences And Management    3039 non-null float64
Citations Per Faculty             7443 non-null float64
Faculty Student                   7524 non-null float64
International Faculty             7386 non-null float64
Academic Reputation               6939 non-null float64
Employer Reputation               7066 non-null float64
Location                          10047 non-null object
NID                               10047 non-null int64
Overall Score                     4694 non-null float64
Region                            10047 non-null objec

In [32]:
qs_df.groupby('Year').agg({
    'Overall Score': lambda x: x.isna().sum(),
    'NID': ['count', 'nunique', 'size']
})

Overall Score   NID              
          <lambda> count nunique  size
Year                                  
2012         369.0   866     865   866
2014         403.0   900     900   900
2015         388.0   885     885   885
2016         515.0   914     914   914
2017         533.0   933     933   933
2018         577.0   977     977   977
2019         515.0  1018    1018  1018
2020         570.0  1069    1069  1069
2021         684.0  1185    1185  1185
2022         799.0  1300    1300  1300

In [33]:
qs_df.head(5)

,City,Natural Sciences,Life Sciences And Medicine,Engineering And Technology,Arts And Humanities,International Students,Social Sciences And Management,Citations Per Faculty,Faculty Student,International Faculty,Academic Reputation,Employer Reputation,Location,NID,Overall Score,Region,University,Year
0,CAMBRIDGE,95.06,60.89,100.00,46.27,96.5,64.62,99.3,99.9,86.4,100.0,100.0,UNITED STATES,294850,100.00,NORTH AMERICA,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),2012
1,CAMBRIDGE,98.50,82.75,66.51,95.21,96.0,72.97,97.0,98.3,98.2,100.0,100.0,UNITED KINGDOM,294561,99.78,EUROPE,UNIVERSITY OF CAMBRIDGE,2012
2,CAMBRIDGE,100.00,100.00,46.14,100.00,78.4,100.00,100.0,98.6,90.0,100.0,100.0,UNITED STATES,294270,99.15,NORTH AMERICA,HARVARD UNIVERSITY,2012
3,LONDON,40.24,49.02,33.13,54.56,99.9,42.15,94.0,98.4,96.3,99.6,95.6,UNITED KINGDOM,294014,98.69,EUROPE,UCL,2012
4,OXFORD,89.84,78.11,52.70,97.66,95.8,76.47,89.4,100.0,98.0,100.0,100.0,UNITED KINGDOM,294654,98.57,EUROPE,UNIVERSITY OF OXFORD,2012


#### What is the duplicate university ('NID') from 2012?

In [34]:
qs_df.loc[
    qs_df.Year==2012,
    'NID'
].value_counts().head(3)

295397    2
296893    1
293446    1
Name: NID, dtype: int64

In [35]:
qs_df.loc[(qs_df.NID==295397)&(qs_df.Year==2012)]

,City,Natural Sciences,Life Sciences And Medicine,Engineering And Technology,Arts And Humanities,International Students,Social Sciences And Management,Citations Per Faculty,Faculty Student,International Faculty,Academic Reputation,Employer Reputation,Location,NID,Overall Score,Region,University,Year
726,PARIS,0.0,0.0,0.0,0.0,97.3,29.12,0.0,0.0,98.4,0.0,98.5,FRANCE,295397,NaN,EUROPE,HEC PARIS SCHOOL OF MANAGEMENT,2012
743,PARIS,0.0,0.0,0.0,0.0,97.3,29.12,0.0,0.0,98.4,0.0,98.5,FRANCE,295397,NaN,EUROPE,HEC PARIS SCHOOL OF MANAGEMENT,2012


In [36]:
# Drop it
qs_df.drop_duplicates(subset=['NID', 'Year'], inplace=True)

In [37]:
qs_df.groupby('Year').agg({
    'Overall Score': lambda x: x.isna().sum(),
    'NID': ['count', 'nunique', 'size']
})

Overall Score   NID              
          <lambda> count nunique  size
Year                                  
2012         368.0   865     865   865
2014         403.0   900     900   900
2015         388.0   885     885   885
2016         515.0   914     914   914
2017         533.0   933     933   933
2018         577.0   977     977   977
2019         515.0  1018    1018  1018
2020         570.0  1069    1069  1069
2021         684.0  1185    1185  1185
2022         799.0  1300    1300  1300

#### RE-CALCULATE OVERALL SCORE

In [38]:
qs_weights = {
    'labs': ['Academic Reputation', 'Employer Reputation', 'Faculty Student', 'Citations Per Faculty', 'International Students', 'International Faculty'],
    2021: [40, 10, 20, 20, 5, 5]
}

In [39]:
qs_df[qs_weights['labs']+['Year']].groupby('Year').apply(
    lambda tx: tx[qs_weights['labs']].isna().any(axis=1).sum()
)

Year
2012      1
2014      0
2015      0
2016    818
2017    793
2018    848
2019    520
2020    571
2021    690
2022     75
dtype: int64

In [40]:
qs_df[qs_weights['labs']].describe()

,Academic Reputation,Employer Reputation,Faculty Student,Citations Per Faculty,International Students,International Faculty
count,6938.000000,7065.00000,7523.000000,7442.000000,7465.000000,7385.000000
mean,34.378221,36.25390,43.384554,36.845834,40.619344,42.596886
std,30.370608,30.80063,31.169069,29.589601,33.407868,36.560410
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,6.800000,6.90000,17.050000,7.700000,8.600000,5.500000
50%,29.300000,32.10000,40.000000,34.400000,35.400000,34.800000
75%,53.275000,57.90000,67.600000,58.100000,68.300000,79.800000
max,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000


In [41]:
sub_df = qs_df[
    qs_weights['labs']
].copy().fillna(0)

#res = np.ma.average(
#    np.ma.array(sub_df, mask=sub_df.isnull()),
#    axis=1,
#    weights=qs_weights.get(2021, qs_weights[2021])
#)

res = np.average(
    sub_df,
    axis=1,
    weights=qs_weights[2021]
)

qs_df.loc[
    :,
    'scores_overall_num'
] = np.round(res.data, 2)


qs_df.loc[:, 'scores_overall_calc'] = qs_df.groupby('Year').apply(
    lambda tx: (tx.scores_overall_num / tx.scores_overall_num.max() * 100).round(2)
).values

In [42]:
# Check calculations disrepancy
qs_df.groupby('Year').apply(
    lambda tx: (tx['Overall Score'] - tx.scores_overall_calc).abs().mean(skipna=True)
)

Year
2012    2.089557
2014    1.665976
2015    2.644044
2016    6.126190
2017    3.516925
2018    3.568700
2019    0.029145
2020    0.025371
2021    0.026347
2022    0.024950
dtype: float64

#### Checking the disrepancies

In [43]:
qs_df.groupby('Year').apply(
    lambda tx: tx[qs_weights['labs']].isna().any(axis=1).sum()
)

Year
2012      1
2014      0
2015      0
2016    818
2017    793
2018    848
2019    520
2020    571
2021    690
2022     75
dtype: int64

In [44]:
qs_df.loc[
    qs_df[qs_weights['labs']].isna().any(axis=1),
    qs_weights['labs'] + ['Year']
].groupby('Year').size()

Year
2012      1
2016    818
2017    793
2018    848
2019    520
2020    571
2021    690
2022     75
dtype: int64

In [45]:
qs_df16 = qs_df[ (qs_df['Year']==2016)]

qs_df16['score_diff'] = (qs_df16['Overall Score'] - qs_df16['scores_overall_calc']).abs()

/home/d2ski/miniconda3/envs/vizlab/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
qs_df16.loc[
    qs_df16.score_diff>1,
    qs_weights['labs'] + ['Overall Score', 'scores_overall_calc', 'score_diff', 'NID']
           ].sort_values(by='score_diff', ascending=False)

,Academic Reputation,Employer Reputation,Faculty Student,Citations Per Faculty,International Students,International Faculty,Overall Score,scores_overall_calc,score_diff,NID
3008,NaN,NaN,NaN,NaN,89.1,94.8,37.6,9.22,28.38,294152
2881,NaN,NaN,NaN,77.9,98.0,97.7,49.3,25.42,23.88,297486
3043,NaN,88.5,NaN,NaN,50.8,32.6,35.5,13.05,22.45,294869
3002,NaN,NaN,78.5,NaN,NaN,NaN,38.0,15.73,22.27,294477
3042,NaN,41.0,NaN,NaN,92.1,93.7,35.5,13.42,22.08,294098
3030,NaN,NaN,70.1,NaN,NaN,NaN,36.1,14.05,22.05,294189
2970,NaN,NaN,84.0,NaN,NaN,35.1,40.4,18.60,21.80,297222
3009,NaN,NaN,47.0,NaN,54.6,73.5,37.5,15.83,21.67,294743
3049,NaN,NaN,67.5,NaN,NaN,NaN,35.0,13.53,21.47,297217
2935,NaN,60.6,72.8,NaN,NaN,37.2,43.7,22.53,21.17,294051


In [47]:
qs_df.head(3)

,City,Natural Sciences,Life Sciences And Medicine,Engineering And Technology,Arts And Humanities,International Students,Social Sciences And Management,Citations Per Faculty,Faculty Student,International Faculty,Academic Reputation,Employer Reputation,Location,NID,Overall Score,Region,University,Year,scores_overall_num,scores_overall_calc
0,CAMBRIDGE,95.06,60.89,100.00,46.27,96.5,64.62,99.3,99.9,86.4,100.0,100.0,UNITED STATES,294850,100.00,NORTH AMERICA,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),2012,98.98,100.00
1,CAMBRIDGE,98.50,82.75,66.51,95.21,96.0,72.97,97.0,98.3,98.2,100.0,100.0,UNITED KINGDOM,294561,99.78,EUROPE,UNIVERSITY OF CAMBRIDGE,2012,98.77,99.79
2,CAMBRIDGE,100.00,100.00,46.14,100.00,78.4,100.00,100.0,98.6,90.0,100.0,100.0,UNITED STATES,294270,99.15,NORTH AMERICA,HARVARD UNIVERSITY,2012,98.14,99.15


#### SAVE QS SCORES

In [48]:
# Save .csv
#qs_df.loc[
#    :,
#    ['Location', 'Region', 'University', 'NID', 'Year', 'scores_overall_calc'] + qs_weights['labs']
#].to_csv(DATASET_OUTPUT_DIR + 'qs_scores.csv')

### MERGE DATA FROM ALL RANKINGS

In [49]:
the_df.head(3)

,location,name,nid,rank,scores_citations,scores_citations_rank,scores_industry_income,scores_industry_income_rank,scores_international_outlook,scores_international_outlook_rank,...,scores_research_rank,scores_teaching,scores_teaching_rank,year,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,scores_overall_num,scores_overall_calc
0,United States,Harvard University,466,1,98.800003,8,34.500000,105,72.400002,49,...,2,99.699997,1,2011,NaN,NaN,NaN,NaN,96.1,96.11
1,United States,California Institute of Technology,128779,2,99.900002,1,83.699997,24,54.599998,93,...,4,97.699997,4,2011,NaN,NaN,NaN,NaN,96.0,96.00
2,United States,Massachusetts Institute of Technology,471,3,99.900002,2,87.500000,21,82.300003,36,...,11,97.800003,3,2011,NaN,NaN,NaN,NaN,95.6,95.53


In [50]:
arwu_df.head(3)

,Alumni,Award,HiCi,N&S,PCP,PUB,ranking,region,regionLogo,regionRanking,score,univNameEn,univUp,year,scores_overall_num,scores_overall_calc
0,98.6,100.0,100.0,100.0,60.6,100.0,1,United States,us,1,100.0,Harvard University,harvard-university,2004,95.92,100.0
1,41.2,72.2,96.1,75.2,68.1,72.3,2,United States,us,2,77.2,Stanford University,stanford-university,2004,74.09,77.2
2,100.0,93.4,56.6,58.5,73.2,70.2,3,United Kingdom,gb,1,76.2,University of Cambridge,university-of-cambridge,2004,73.06,76.2


In [51]:
qs_df.head(3)

,City,Natural Sciences,Life Sciences And Medicine,Engineering And Technology,Arts And Humanities,International Students,Social Sciences And Management,Citations Per Faculty,Faculty Student,International Faculty,Academic Reputation,Employer Reputation,Location,NID,Overall Score,Region,University,Year,scores_overall_num,scores_overall_calc
0,CAMBRIDGE,95.06,60.89,100.00,46.27,96.5,64.62,99.3,99.9,86.4,100.0,100.0,UNITED STATES,294850,100.00,NORTH AMERICA,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),2012,98.98,100.00
1,CAMBRIDGE,98.50,82.75,66.51,95.21,96.0,72.97,97.0,98.3,98.2,100.0,100.0,UNITED KINGDOM,294561,99.78,EUROPE,UNIVERSITY OF CAMBRIDGE,2012,98.77,99.79
2,CAMBRIDGE,100.00,100.00,46.14,100.00,78.4,100.00,100.0,98.6,90.0,100.0,100.0,UNITED STATES,294270,99.15,NORTH AMERICA,HARVARD UNIVERSITY,2012,98.14,99.15


#### ADD COUNTRY COLUMN

In [52]:
# Unify country names
countries = np.union1d(
    the_df.location.str.lower(),
    np.union1d(arwu_df.region.str.lower(), qs_df.Location.str.lower())
)

failed = []
country_map = {}

for loc in countries:
    try:
        country = pycountry.countries.search_fuzzy(loc)
        country_map[loc] = country[0].name
    except LookupError as e:
        failed.append(loc)

In [53]:
failed

['china (mainland)',
 'china-hong kong',
 'china-macau',
 'china-taiwan',
 'macau sar',
 'northern cyprus',
 'palestinian territory, occupied',
 'south korea']

In [54]:
failed_add = [
    'China',
    'Hong Kong',
    'Macao',
    'Taiwan, Province of China',
    'Macao',
    'Cyprus',
    'Palestine, State of',
    'Korea, Republic of'
]

for loc, country in zip(failed, failed_add):
    country_map[loc] = country
    
country_map['hong kong sar'] = 'Hong Kong'

In [55]:
the_df.loc[:, 'Country'] = the_df.location.str.lower().apply(lambda x: country_map[x])
arwu_df.loc[:, 'Country'] = arwu_df.region.str.lower().apply(lambda x: country_map[x])
qs_df.loc[:, 'Country'] = qs_df.Location.str.lower().apply(lambda x: country_map[x])

#### MERGE

In [56]:
the_scores = the_df[['Country', 'year', 'scores_overall_num', 'name']].copy().rename(
    columns = {
    'year': 'Year',
    'scores_overall_num': 'Score',
    'name': 'University',
})

the_scores.loc[:, 'Source'] = 'THE'

In [57]:
arwu_scores = arwu_df[['Country', 'year', 'scores_overall_calc', 'univNameEn']].copy().rename(
    columns = {
    'year': 'Year',
    'scores_overall_calc': 'Score',
    'univNameEn': 'University',
})

arwu_scores.loc[:, 'Source'] = 'ARWU'

arwu_scores.loc[:, 'Year'] = arwu_scores.loc[:, 'Year'] + 1

In [58]:
qs_scores = qs_df[['Country', 'Year', 'scores_overall_calc', 'University']].copy().rename(
    columns = {
    'scores_overall_calc': 'Score',
})

qs_scores.loc[:, 'Source'] = 'QS'

#### MERGE AND SUBSET TOP-500 UNIVERSITIES BY SCORE RANK IN EVERY RANKING

In [108]:
scores = pd.concat([
    the_scores,
    arwu_scores,
    qs_scores
], ignore_index=True)

In [109]:
scores.loc[
    :,
    'Rank'
] = scores.groupby(['Source', 'Year'])['Score'].rank(method='first', ascending=False)

In [110]:
scores.head(10)

,Country,Year,Score,University,Source,Rank
0,United States,2011,96.1,Harvard University,THE,1.0
1,United States,2011,96.0,California Institute of Technology,THE,2.0
2,United States,2011,95.6,Massachusetts Institute of Technology,THE,3.0
3,United States,2011,94.3,Stanford University,THE,4.0
4,United States,2011,94.2,Princeton University,THE,5.0
5,United Kingdom,2011,91.2,University of Oxford,THE,6.0
6,United Kingdom,2011,91.2,University of Cambridge,THE,7.0
7,United States,2011,91.1,"University of California, Berkeley",THE,8.0
8,United Kingdom,2011,90.6,Imperial College London,THE,9.0
9,United States,2011,89.5,Yale University,THE,10.0


In [111]:
# Data from 2014 only
scores = scores.loc[ scores.Year >= 2014]

In [112]:
# Score stats
scores.groupby(['Year', 'Source']).agg({
    'Score': 'sum',
    'University': 'count',
    'Rank': 'max'
})

Score  University    Rank
Year Source                              
2014 ARWU     9338.70         500   500.0
     QS      27668.50         900   900.0
     THE     18850.29         400   400.0
2015 ARWU     9415.80         500   500.0
     QS      28215.89         885   885.0
     THE     19521.72         401   401.0
2016 ARWU     9386.50         500   500.0
     QS      26058.77         914   914.0
     THE     30501.37         800   800.0
2017 ARWU     9626.50         500   500.0
     QS      25099.06         933   933.0
     THE     35438.67         981   981.0
2018 ARWU    12437.80         800   800.0
     QS      25464.83         977   977.0
     THE     39348.04        1103  1103.0
2019 ARWU    13808.70        1000  1000.0
     QS      27596.56        1018  1018.0
     THE     44226.02        1258  1258.0
2020 ARWU    14181.20        1000  1000.0
     QS      27695.67        1069  1069.0
     THE     48587.43        1397  1397.0
2021 ARWU    14160.20         999   999.0
     QS      27981.62        1185  1185.0
     THE     52697.47        1526  1526.0
2022 ARWU    14426.40        1000  1000.0
     QS      32772.09        1300  1300.0
     THE     56959.69        2112  2112.0

In [131]:
# TOP-500 universities only
scores500 = scores.loc[
    scores.Rank <= 500
]

In [155]:
# Aggregate by country
country_scores500 = scores500.groupby(['Country', 'Year', 'Source'])

country_scores500_data = pd.DataFrame({
    'RankPct': country_scores500['Rank'].count() / country_scores500['Rank'].max() * 100,
    'RankCount': country_scores500['Rank'].count(),
    'RankMean': country_scores500['Rank'].mean(),
    'RankMedian': country_scores500['Rank'].median(),
}).reset_index().round(0)

In [157]:
# Check USA
country_scores500_data.loc[
    country_scores500_data.Country == 'United States'
].sort_values(by=['Source', 'Year'])

,Country,Year,Source,RankPct,RankCount,RankMean,RankMedian
1305,United States,2014,ARWU,30.0,149,195.0,172.0
1308,United States,2015,ARWU,30.0,146,202.0,178.0
1311,United States,2016,ARWU,30.0,146,204.0,185.0
1314,United States,2017,ARWU,28.0,137,199.0,186.0
1317,United States,2018,ARWU,27.0,135,199.0,188.0
1320,United States,2019,ARWU,28.0,139,213.0,207.0
1323,United States,2020,ARWU,28.0,137,214.0,215.0
1326,United States,2021,ARWU,27.0,133,212.0,204.0
1329,United States,2022,ARWU,26.0,129,211.0,205.0
1306,United States,2014,QS,21.0,104,225.0,201.0


#### SAVE DATA FOR VISUALIZATION

In [158]:
scores500.to_csv(VIZ_DATA_DIR + 'scores500.csv', index=False)
country_scores500_data.to_csv(VIZ_DATA_DIR + 'country_scores500.csv', index=False)